In [1]:
import numpy as np
import datetime as dt
import pandas as pd
import yfinance as yf

In [2]:
last_day = dt.datetime.now().date()
ggal_df = yf.download('GGAL', 
                      start='2009-12-10', 
                      end=last_day, 
                      progress=False)
ggal_df

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2009-12-10,5.30,5.46,5.20,5.31,5.030039,237300
2009-12-11,5.44,5.48,5.35,5.48,5.191077,48200
2009-12-14,5.53,5.64,5.31,5.63,5.333169,87300
2009-12-15,5.55,5.75,5.47,5.70,5.399477,119200
2009-12-16,5.68,5.80,5.53,5.75,5.446842,106600
...,...,...,...,...,...,...
2020-11-09,8.15,8.69,8.09,8.09,8.090000,1511300
2020-11-10,8.26,8.92,8.02,8.80,8.800000,2131700
2020-11-11,8.75,8.78,8.28,8.30,8.300000,1158500


In [3]:
ggal_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2752 entries, 2009-12-10 to 2020-11-13
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Open       2752 non-null   float64
 1   High       2752 non-null   float64
 2   Low        2752 non-null   float64
 3   Close      2752 non-null   float64
 4   Adj Close  2752 non-null   float64
 5   Volume     2752 non-null   int64  
dtypes: float64(5), int64(1)
memory usage: 150.5 KB


In [4]:
ggal_df['ma'] = ggal_df['Close'].rolling('21d').mean()
ggal_df['pulse'] = np.where((ggal_df['Close']-ggal_df['ma'])>0,1,0)
ggal_df['cross'] = ggal_df['pulse'].diff() #venta -1, #compra 1

In [5]:
fil = (ggal_df['cross']!=0)&(ggal_df.index-ggal_df.index[0]>'21d')
profit = ggal_df[fil]

In [6]:
profit.head(10) #operaciones: cross, -1 venta, 1 compra

,Open,High,Low,Close,Adj Close,Volume,ma,pulse,cross
Date,,,,,,,,,
2010-01-11,5.86,5.86,5.41,5.51,5.219495,145600,5.646923,0,-1.0
2010-02-12,4.84,5.15,4.83,5.15,4.878475,87100,5.087333,1,1.0
2010-02-16,5.17,5.17,4.80,4.92,4.660602,123100,5.052143,0,-1.0
2010-03-01,4.76,5.00,4.76,4.97,4.707965,79800,4.870000,1,1.0
2010-04-01,5.99,5.99,5.75,5.83,5.522623,81400,5.853333,0,-1.0
2010-04-05,5.85,6.20,5.85,6.09,5.768916,178800,5.942143,1,1.0
2010-04-16,6.34,6.34,6.00,6.07,5.749970,67600,6.136429,0,-1.0
2010-04-20,6.22,6.23,6.14,6.16,5.835225,62200,6.143571,1,1.0
2010-04-27,6.23,6.35,6.00,6.08,5.759442,231900,6.230000,0,-1.0


In [7]:
# comienzo con una compra
if profit.iloc[0]['cross'] != 1.0:
    profit = profit[1:]

if profit.iloc[0]['cross'] != 1.0:
    profit = profit[1:]

# termino con una venta
if profit.iloc[-1]['cross'] != -1.0:
    profit = profit[:-1]


In [8]:
start_points, end_points = profit.iloc[::2], profit.iloc[1::2]
start_points = start_points.reset_index()
end_points = end_points.reset_index()
print(start_points.shape)
print(end_points.shape)

(176, 10)
(176, 10)


In [9]:
transa = pd.DataFrame()
# Compras
transa['buy'] = start_points['Close']
transa['buy_date'] = start_points['Date']

# Ventas
transa['sell'] = end_points['Close']
transa['sell_date'] = end_points['Date']

transa['profit']=transa['sell']-transa['buy']
transa['cumprofit'] = transa['profit'].cumsum()

In [10]:
# primeras operaciones
transa.head()

,buy,buy_date,sell,sell_date,profit,cumprofit
0,5.15,2010-02-12,4.92,2010-02-16,-0.23,-0.23
1,4.97,2010-03-01,5.83,2010-04-01,0.86,0.63
2,6.09,2010-04-05,6.07,2010-04-16,-0.02,0.61
3,6.16,2010-04-20,6.08,2010-04-27,-0.08,0.53
4,5.55,2010-05-28,5.78,2010-06-29,0.23,0.76


In [11]:
# últimas operaciones
transa.tail()

,buy,buy_date,sell,sell_date,profit,cumprofit
171,10.22,2020-06-19,9.85,2020-06-22,-0.37,44.860007
172,10.67,2020-06-25,10.21,2020-06-26,-0.46,44.400007
173,10.33,2020-06-29,9.70,2020-06-30,-0.63,43.770007
174,11.32,2020-07-06,12.44,2020-08-10,1.12,44.890007
175,10.26,2020-09-08,9.74,2020-09-11,-0.52,44.370007


In [12]:
# ganancia acumulada
transa['profit'].sum()

44.3700065612793

In [13]:
import plotly.graph_objects as go
import plotly.offline as plt
from plotly.subplots import make_subplots

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=ggal_df.index, y=ggal_df['Close'], 
                         name='Precio', mode='lines', line=dict(color='royalblue', width=2))
             )

fig.add_trace(go.Scatter(x=ggal_df.index, y=ggal_df['ma'], 
                         name='MA-21días', mode='lines', line=dict(color='red', width=2),
                         opacity = 0.5
                        )
             )

fig.add_trace(go.Scatter(x=transa['sell_date'], y=transa['cumprofit'], 
                         name='Ganancia Acumulada', mode='lines', line=dict(color='green', width=3),
                         opacity = 0.8
                        )
             )



fig.update_layout(
    title=go.layout.Title(
        text="Bot Alexis MA21. Ganancia acumulada con una acción de GGAL en 10 años = u$s {0:.2f}. En {1} operaciones sin comisión".format(transa['profit'].sum(),len(transa)),
        #xref="paper",
        x=0
    ),
    yaxis=dict(
    title="Dolares [u$s]",
    ),
    xaxis=dict(
    title="Fecha (temporalidad diaria)",
    ),
 )

fig.show()
